In [1]:
import os
import pickle
import pandas as pd
import numpy as np
import nibabel as nib
import pcntoolkit as ptk 
from pcntoolkit.dataio.fileio import load as ptkload
from pcntoolkit.dataio.fileio import save as ptksave
from pcntoolkit.dataio.fileio import save_nifti, load_nifti
from pcntoolkit.normative import predict, evaluate
from pcntoolkit.util.utils import compute_MSLL, create_design_matrix, calibration_descriptives

from scipy import stats

from matplotlib import pyplot as plt
import seaborn as sns
import pingouin as pg

#%% SET GLOBALS
# globals
root_dir = '/project_cephfs/3022017.06/ENIGMA_ANX/'

###  CHANGE DEPENDING ON Z-STAT OR SCALED EFFECT  ###
proc_dir = os.path.join(root_dir,'Z_stat/')
#proc_dir = os.path.join(root_dir,'Scaled_effect/')


data_dir = os.path.join(proc_dir,'data/')
w_dir = os.path.join(proc_dir,'vox/')
mask_nii = ('/opt/fmriprep/templateflow/tpl-MNI152NLin2009cAsym/tpl-MNI152NLin2009cAsym_res-02_desc-brain_mask.nii.gz')
ex_nii = os.path.join(data_dir, 'ENIGMA_FC_tr_1.nii.gz')


/home/preclineu/hansav/.conda/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/preclineu/hansav/.conda/envs/py38/lib/python3.8/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(
/home/preclineu/hansav/.conda/envs/py38/lib/python3.8/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.1, the latest is 0.5.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [2]:
#%% THRESHOLD THE NII FILES USING FSLMATHS
in_filename = (w_dir +'Z_ref.nii.gz')
#print(filename)
out_filename_pos = (w_dir +'Z_ref_pos2pt6.nii.gz')
out_filename_neg = (w_dir +'Z_ref_neg2pt6.nii.gz')
#print(out_filename_pos, out_filename_neg)

command = ('fslmaths ' +in_filename +' -thr 2.6 ' +out_filename_pos)
print(command)
os.system(command)
!command

command = ('fslmaths ' +in_filename +' -mul -1 -thr 2.6 ' +out_filename_neg)
print(command)
os.system(command)
!command

print('THRESHOLD THE NII FILES USING FSLMATHS DONE')

fslmaths /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Z_ref.nii.gz -thr 2.6 /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Z_ref_pos2pt6.nii.gz
fslmaths /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Z_ref.nii.gz -mul -1 -thr 2.6 /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Z_ref_neg2pt6.nii.gz
THRESHOLD THE NII FILES USING FSLMATHS DONE


In [3]:
#%% BINARISE THE NII FILES USING FSLMATHS
in_filename_pos = (w_dir +'Z_ref_pos2pt6.nii.gz')
in_filename_neg = (w_dir +'Z_ref_neg2pt6.nii.gz')
#print(filename)
out_filename_pos = (w_dir +'Z_ref_pos2pt6_bin.nii.gz')
out_filename_neg = (w_dir +'Z_ref_neg2pt6_bin.nii.gz')
#print(out_filename_pos, out_filename_neg)

command = ('fslmaths ' +in_filename_pos +' -bin  ' +out_filename_pos)
print(command)
os.system(command)
!command

command = ('fslmaths ' +in_filename_neg +' -bin  ' +out_filename_neg)
print(command)
os.system(command)
!command

print('BINARISE THE NII FILES USING FSLMATHS DONE')

fslmaths /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Z_ref_pos2pt6.nii.gz -bin  /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Z_ref_pos2pt6_bin.nii.gz
fslmaths /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Z_ref_neg2pt6.nii.gz -bin  /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Z_ref_neg2pt6_bin.nii.gz
BINARISE THE NII FILES USING FSLMATHS DONE


In [4]:
#%% SUM THE NUMBER OF VOXELS
in_filename_pos = (w_dir +'Z_ref_pos2pt6_bin.nii.gz')
in_filename_neg = (w_dir +'Z_ref_neg2pt6_bin.nii.gz')
thresh_bin_nii_pos = load_nifti(in_filename_pos, mask=mask_nii)
thresh_bin_nii_neg = load_nifti(in_filename_neg, mask=mask_nii)

out_filename_pos = (w_dir +'Z_ref_count_pos.nii.gz')
out_filename_neg = (w_dir +'Z_ref_count_neg.nii.gz')
#print(out_filename_pos, out_filename_neg)
   
#Sum along the 2nd dimension(4D = volumes = participants) of the Positive thresholded, binarised file
sum_site_pos = np.sum(thresh_bin_nii_pos,axis=1)
#Save as nii
save_nifti(sum_site_pos,out_filename_pos, examplenii=mask_nii, mask=mask_nii, dtype='float32')

#Sum along the 2nd dimension(4D = volumes = participants) of the Negative thresholded, binarised file
sum_site_neg = np.sum(thresh_bin_nii_neg,axis=1)
#Save as nii
save_nifti(sum_site_neg,out_filename_neg, examplenii=mask_nii, mask=mask_nii, dtype='float32')

print('SUM THE NUMBER OF VOXELS DONE')

SUM THE NUMBER OF VOXELS DONE
